<a href="https://colab.research.google.com/github/sravanisasu/10k-sample/blob/main/WNN_pytorch_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Required imports**

In [124]:
# General
import pandas as pd
import numpy as np
import time
import os
# PyTorch
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm
from torch.utils.data import Dataset, DataLoader
# Scikit Learn
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

np.random.seed(2)

**Morlet Function**

In [125]:
# Morlet Function
def w_func(t):
  return torch.cos(1.75*t)* torch.exp(-(t**2))

**Intialising Hyper parameters and constants**

In [126]:
batch_size= 32
lr = 0.001
mom = 0.9
epochs=100
filename = '/content/Dataset_spine.csv'
data = pd.read_csv(filename)
nin = len(data.columns)-1 # Size of input layer (sample size, number of features) 
nhn = 10 # Size of hidden layer
non = 1

**Model definition**

In [127]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = weight_norm(nn.Linear(nin, nhn, bias=False))
        self.fc2 = weight_norm(nn.Linear(nhn, non, bias=False))
        self.a = nn.Parameter(torch.rand(nhn), requires_grad=True)
        self.b = nn.Parameter(torch.rand(nhn), requires_grad=True)

    def forward(self, x):
        t = (self.fc1(x)-self.b)/self.a
        vk = self.fc2(w_func(t))
        return vk



**Preprocessing dataset**

In [128]:
class CustomDataset(Dataset):
    def __init__(self, x,y):
        self.data = []
        for i,j in zip(x,y):
            self.data.append((i,j))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    @classmethod 
    def splits(cls, filename):
        
        data = pd.read_csv(filename)
        cols_name = data.loc[:].columns
        nonumeric_cols = data.loc[:, data.dtypes == object].columns
        
        #replace empty values with NaN
        data = data.replace("?","NaN")
        
        #convert non numeric columns to numeric values
        for col_name in nonumeric_cols:
            list_total = list(data[col_name])
            list_uniq = list(set((list_total)))
            data[col_name] = [list_uniq.index(data) for data in list_total]
        
        #replace null values with mean
        for col_name in cols_name:
          data[col_name].fillna(data[col_name].mean())

        # Normalizing Data
        data[data.columns] = minmax_scale(data[data.columns])
        
        #extract x and y values from data
        data_pre = np.array(data)
        x=data_pre[:,:-1]
        y=data_pre[:,-1]

        #split the data into train and test and convert to torch
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
        y_train=  torch.from_numpy(np.expand_dims(y_train, axis=1)).float()
        y_test=  torch.from_numpy(np.expand_dims(y_test, axis=1)).float()
        x_train =  torch.from_numpy(x_train).float()
        x_test =  torch.from_numpy(x_test).float()
        
        train = cls(x_train, y_train)
        test = cls(x_test, y_test)
        return train, test

**Average Calculation**

In [129]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

**Training**

In [130]:
#start time
start_time = time.time()

#intialiaze dataset, model, optimiser and losses
model = Net()
loss = 0
losses = AverageMeter()
total_loss = 0
optimizer = torch.optim.SGD(model.parameters(), lr = lr, momentum=mom)
train, test = CustomDataset.splits(filename)
loss_array = []

#dataloader
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=True, num_workers=2)

#train model
model.train()

for epoch in range(epochs):

    #check for exsisting checkpoint
    if os.path.exists("best.pt"):
      checkpoint = torch.load("best.pt")
      best_loss = checkpoint['loss']
    else:
      best_loss = 9999

    for x,y in train_loader:
        if torch.cuda.is_available():
            x = x.to('cuda')
            y = y.to("cuda")
            model.to('cuda')
        optimizer.zero_grad()

        #forward propogation and find losses (predict)
        pred = model(x)
        loss = nn.MSELoss()(pred, y)
        total_loss += loss

        #back propogation
        loss.backward()
        optimizer.step()
        losses.update(loss.item())
    loss_array.append(losses.val)

    #update checkpoint if loss is less than exsisting checkpoint
    if losses.val < best_loss:
      torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': losses.val,
      }, "best.pt")
      
    print_str = 'Epoch: [{0}]\tLoss= {loss.val:.6f} Loss Avg: ({loss.avg:.6f})\t' .format(epoch, loss=losses) 
    print(print_str)

#end time
end_time = time.time()

print('\n')
print("Time taken for the run",end_time-start_time)

Epoch: [0]	Loss= 0.578545 Loss Avg: (0.727406)	
Epoch: [1]	Loss= 0.500047 Loss Avg: (0.690307)	
Epoch: [2]	Loss= 0.689166 Loss Avg: (0.665864)	
Epoch: [3]	Loss= 0.552855 Loss Avg: (0.648098)	
Epoch: [4]	Loss= 0.538248 Loss Avg: (0.634158)	
Epoch: [5]	Loss= 0.508258 Loss Avg: (0.621415)	
Epoch: [6]	Loss= 0.473061 Loss Avg: (0.608243)	
Epoch: [7]	Loss= 0.440325 Loss Avg: (0.594089)	
Epoch: [8]	Loss= 0.414626 Loss Avg: (0.578199)	
Epoch: [9]	Loss= 0.331204 Loss Avg: (0.560364)	
Epoch: [10]	Loss= 0.333444 Loss Avg: (0.541452)	
Epoch: [11]	Loss= 0.277338 Loss Avg: (0.522082)	
Epoch: [12]	Loss= 0.296836 Loss Avg: (0.503266)	
Epoch: [13]	Loss= 0.240196 Loss Avg: (0.485770)	
Epoch: [14]	Loss= 0.226850 Loss Avg: (0.469586)	
Epoch: [15]	Loss= 0.269542 Loss Avg: (0.455096)	
Epoch: [16]	Loss= 0.260897 Loss Avg: (0.442014)	
Epoch: [17]	Loss= 0.251613 Loss Avg: (0.430205)	
Epoch: [18]	Loss= 0.268411 Loss Avg: (0.419601)	
Epoch: [19]	Loss= 0.259806 Loss Avg: (0.409982)	
Epoch: [20]	Loss= 0.225538 Los

**Evaluate model and predict values**

In [131]:
#pick the best model from checkpoint
if os.path.exists("best.pt"):
  checkpoint = torch.load("best.pt")
  model.load_state_dict(checkpoint['model_state_dict'])

#evaluate model
model.eval()
test_loss = 0
losses = AverageMeter()
y_true = []
y_pred = []

for x,y in test_loader:

    if torch.cuda.is_available():
        x = x.to('cuda')
        y = y.to("cuda")
    with torch.no_grad():

      #predict values and find losses
      pred = model(x)
      test_loss += nn.MSELoss()(pred, y)
      losses.update(loss.item())
      y = y.cpu()
      pred = pred.cpu()

      #store predicted and true values into lists
      for y_d in y:
        y_true.append(y_d.item())
      for pred_d in pred:
        y_pred.append(pred_d.item())

print_str = 'Loss: {loss.val:.6f} Loss Avg: ({loss.avg:.6f})\t'.format(loss=losses) 
print(print_str)

Loss: 0.164552 Loss Avg: (0.164552)	


In [132]:
mape = mean_absolute_error(y_true, y_pred)*100
print(mape)

37.0305938586112


In [133]:
rm /content/best.pt